In [1]:
import logging

import numpy

import cicada.additive
import cicada.communicator

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)

    base = numpy.array([-1, 2, 3.4, -2.3]) if communicator.rank == 0 else None
    exponent = 3
    
    log.info(f"Player {communicator.rank} base: {base}", src=0)
    log.info(f"Player {communicator.rank} exponent: {exponent}")
    
    base_share = protocol.share(src=0, secret=protocol.encoder.encode(base), shape=(4,))
    power_share = protocol.private_public_power(base_share, exponent)
    power = protocol.encoder.decode(protocol.reveal(power_share))
    log.info(f"Player {communicator.rank} power: {power}")
    
main();

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:58709 from tcp://127.0.0.1:58709.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:58709 from tcp://127.0.0.1:58710.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:58709 from tcp://127.0.0.1:58711.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 base: [-1.   2.   3.4 -2.3]
INFO:root:Player 0 exponent: 3
INFO:root:Player 1 exponent: 3
INFO:root:Player 2 exponent: 3
INFO:root:Player 0 power: [ -1.           8.          39.30377197 -12.16680908]
INFO:root:Player 1 power: [ -1.           8.          39.30377197 -12.16680908]
INFO:root:Player 2 power: [ -1.           8.          39.30377197 -12.16680908]
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator

Note that the values 39.30381775 and -12.16680908 are slightly off due to the limited precision of our fixed point encoding.